In [68]:
import pickle
import numpy as np
import os
from collections import defaultdict

# --- Settings ---
window_size = 30     # in seconds
stride = 5       # in seconds

for y in range(1,2):
    for x in range(1,11):
        # num_iterations = x 
        # window_size = 10 +(x-1) * 5

        Astart = 562  # anomaly start in seconds from 18:30:00 (18:39:22)
        Aend = 752    # anomaly end in seconds from 18:30:00 (18:42:32)
        Ttotal = 14 * 60  # total duration in seconds (840)

        total_windows = (Ttotal - window_size) // stride + 1

        # Calculate true anomaly window count
        true_anomaly_count = max(0, (Aend // stride) - ((Astart - window_size) // stride) + 1)

        # Percentile threshold for detection
        PERCENTILE = 100 - (true_anomaly_count / total_windows) * 100 if total_windows > 0 else 0

        # --- Load Results ---
        with open(f'results/RS_500/ensemble_res_{x}.pkl', 'rb') as f:
            all_results = pickle.load(f)

        # --- Compute Anomaly Scores ---
        anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

        for iteration_result in all_results:
            buckets = iteration_result['buckets']
            bucket_results = iteration_result['bucket_results']

            for bucket_result in bucket_results:
                bucket_idx = bucket_result['bucket_idx']
                final_results = bucket_result['final_results']
                indices_in_bucket = buckets[bucket_idx]

                mean = np.mean(final_results)
                std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

                for i, idx in enumerate(indices_in_bucket):
                    sim = final_results[i]
                    deviation = abs(sim - mean) / std
                    anomaly_scores[idx]['score_sum'] += deviation
                    anomaly_scores[idx]['count'] += 1

        # --- Final Score Calculation ---
        final_scores = {
            idx: score_data['score_sum'] / score_data['count']
            for idx, score_data in anomaly_scores.items()
        }

        # --- Compute Threshold by Percentile ---
        all_score_values = list(final_scores.values())
        threshold = np.percentile(all_score_values, PERCENTILE)

        # --- Flag Anomalous Windows ---
        detected_windows = {
            idx: score for idx, score in final_scores.items() if score >= threshold
        }

        # --- Ground Truth: Actual Anomalous Windows ---
        start_times = [i * stride for i in range(total_windows)]
        anomalous_indices = set()
        for idx, start in enumerate(start_times):
            end = start + window_size
            if min(end, Aend) > max(start, Astart):  # overlap exists
                anomalous_indices.add(idx)

        # --- Detected Indices ---
        detected_indices = set(detected_windows.keys())

        # --- Metrics ---
        true_positives = len(detected_indices & anomalous_indices)
        false_positives = len(detected_indices - anomalous_indices)
        false_negatives = len(anomalous_indices - detected_indices)

        precision = true_positives / len(detected_indices) * 100 if detected_indices else 0
        recall = true_positives / len(anomalous_indices) * 100 if anomalous_indices else 0
        f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

        # --- Output ---
        print(f"\nDetected {len(detected_windows)} anomalous windows above the {PERCENTILE:.2f}th percentile (threshold ≈ {threshold:.4f}):\n")
        for idx, score in sorted(detected_windows.items()):
            print(f"Window {idx} - Score: {score:.4f}")

        print("\n--- Detection Metrics ---")
        print(f"True Positives: {true_positives}")
        print(f"False Positives: {false_positives}")
        print(f"False Negatives: {false_negatives}")
        print(f"Precision: {precision:.2f}%")
        print(f"Recall: {recall:.2f}%")
        print(f"F1 Score: {f1_score:.2f}%")

        # --- Save Results ---
        output_dir = f"evaluations/RS_500/"
        os.makedirs(output_dir, exist_ok=True)  # Create folder if it doesn't exist
        output_filename = f"results_w{x}.txt"
        output_path = os.path.join(output_dir, output_filename)

        with open(output_path, "w") as f:
            for idx, score in sorted(detected_windows.items()):
                f.write(f"Window {idx} - Score: {score:.4f}\n")
            f.write(f"\nTrue Positives: {true_positives}")
            f.write(f"\nFalse Positives: {false_positives}")
            f.write(f"\nFalse Negatives: {false_negatives}")
            f.write(f"\nPrecision: {precision:.2f}%")
            f.write(f"\nRecall: {recall:.2f}%")
            f.write(f"\nF1 Score: {f1_score:.2f}%")

        print("\nSaved to 'resssssssss'")


Detected 43 anomalous windows above the 72.39th percentile (threshold ≈ 0.8658):

Window 5 - Score: 0.8778
Window 6 - Score: 0.8705
Window 7 - Score: 0.8666
Window 8 - Score: 0.9128
Window 10 - Score: 0.9153
Window 25 - Score: 0.8694
Window 46 - Score: 0.8675
Window 106 - Score: 0.8844
Window 107 - Score: 0.8754
Window 108 - Score: 0.9711
Window 109 - Score: 0.9033
Window 110 - Score: 0.8912
Window 111 - Score: 0.9358
Window 112 - Score: 0.9159
Window 113 - Score: 0.9298
Window 114 - Score: 0.9356
Window 115 - Score: 0.9055
Window 116 - Score: 0.9122
Window 117 - Score: 0.9032
Window 118 - Score: 0.8804
Window 120 - Score: 0.8899
Window 121 - Score: 0.8931
Window 122 - Score: 0.8836
Window 123 - Score: 0.8686
Window 124 - Score: 0.9165
Window 125 - Score: 0.8938
Window 126 - Score: 0.9079
Window 127 - Score: 0.8964
Window 128 - Score: 0.8685
Window 129 - Score: 0.9437
Window 130 - Score: 0.9096
Window 131 - Score: 0.8966
Window 132 - Score: 0.9484
Window 133 - Score: 0.9376
Window 134

[Error] Could not parse results/EC_times/1\slurm-1228704_1.out
[Error] Could not parse results/EC_times/1\slurm-1228704_2.out
[Error] Could not parse results/EC_times/1\slurm-1228704_3.out
[Error] Could not parse results/EC_times/1\slurm-1228704_4.out
[Error] Could not parse results/EC_times/1\slurm-1228704_5.out
[Error] Could not parse results/EC_times/1\slurm-1228704_6.out
[Error] Could not parse results/EC_times/1\slurm-1228704_7.out
[Error] Could not parse results/EC_times/1\slurm-1228704_8.out
[Error] Could not parse results/EC_times/1\slurm-1228704_9.out
[Error] Could not parse results/EC_times/1\slurm-1228704_10.out
[Error] Could not parse results/EC_times/1\slurm-1228704_11.out
[Error] Could not parse results/EC_times/1\slurm-1228704_12.out
[Error] Could not parse results/EC_times/1\slurm-1228704_13.out
[Error] Could not parse results/EC_times/1\slurm-1228704_14.out
[Error] Could not parse results/EC_times/1\slurm-1228704_15.out
[Error] Could not parse results/EC_times/2\slurm-